<a href="https://colab.research.google.com/github/doSeung11/2023-2-CECD3-Release-8/blob/main/JangguSound75.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import os
import librosa
import librosa.display
import numpy as np

In [ ]:
!sudo pip install numba
!sudo pip install llvmlite
!sudo pip install librosa
!sudo apt-get install ffmpeg
!sudo pip install resampy
!sudo pip install keras
!sudo pip install np_utils

#한글 깨짐 방지
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [20]:
import sklearn.preprocessing
max_ped_len = 75
#maxlen = 0

def extract_features(file_name):

    try:
        wav, sample_rate = librosa.load(file_name, res_type='kaiser_fast')
        mfcc = librosa.feature.mfcc(y=wav, sr=sample_rate, n_mfcc=100)

        #global maxlen
        #if mfcc.shape[1] > maxlen: maxlen = mfcc.shape[1]
        #print(maxlen)

        pad_width = max_ped_len - mfcc.shape[1]
        mfcc = np.pad(mfcc, pad_width=((0,0), (0,pad_width)),mode='constant')

    except Exception as e:
        print("Error encountered: ", e)
        return None

    return mfcc

fulldatasetpath = '/content/drive/MyDrive/CSE/newnewset/'
features = []

#Iterate through each sound file and extract the features
for file_name in os.listdir(fulldatasetpath):

    if 'wav' not in file_name:
      continue

    data = extract_features(fulldatasetpath+file_name)
    class_label = file_name[0:3]

    features.append([data, class_label])

#Convert into a Panda dataframe
featuresdf = pd.DataFrame(features, columns=['feature', 'class_label'])

print('Finished feature extraction from ', len(featuresdf), ' files')
#print(maxlen)

Finished feature extraction from  900  files


In [21]:
#학습 데이터와 훈련 데이터 분할
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

#Convert features and corresponding classification labels into numpy arrays
X = np.array(featuresdf.feature.tolist())

y = np.array(featuresdf.class_label.tolist())

#Encode the classification labels
le = LabelEncoder()
yy = to_categorical(le.fit_transform(y))

#Split the dataset
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, yy, test_size=0.2, random_state = 42)

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(720, 100, 75)
(180, 100, 75)
(720, 3)
(180, 3)


In [23]:
#CNN model
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.optimizers import Adam
from sklearn import metrics

num_rows = 100
num_columns = 75
num_channel = 1

x_train = x_train.reshape(x_train.shape[0], num_rows, num_columns, num_channel)
x_test = x_test.reshape(x_test.shape[0], num_rows, num_columns, num_channel)
print(x_train.shape)
print(x_test.shape)

num_labels = yy.shape[1]
filter_size = 2

# Construct model
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=2, input_shape=(x_train.shape[1],  x_train.shape[2], num_channel), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=32, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=64, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=128, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))
model.add(GlobalAveragePooling2D())


model.add(Dense(num_labels, activation='softmax'))

(720, 100, 75, 1)
(180, 100, 75, 1)


In [40]:
#모델 컴파일
# Compile the model
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

# Display model architecture summary
model.summary()


#학습 데이터로 모델 훈련
from keras.callbacks import ModelCheckpoint
from datetime import datetime

num_epochs = 200
num_batch_size = 45

#checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.basic_cnn.hdf5', verbose=1, save_best_only=True)

start = datetime.now()

history = model.fit(x_train, y_train,
                    batch_size = num_batch_size,
                    epochs = num_epochs,
                    validation_data = (x_test, y_test),
                    verbose = 1)

duration = datetime.now() - start
print("Training completed in time: ", duration)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 99, 74, 16)        80        
                                                                 
 max_pooling2d_8 (MaxPoolin  (None, 49, 37, 16)        0         
 g2D)                                                            
                                                                 
 dropout_8 (Dropout)         (None, 49, 37, 16)        0         
                                                                 
 conv2d_9 (Conv2D)           (None, 48, 36, 32)        2080      
                                                                 
 max_pooling2d_9 (MaxPoolin  (None, 24, 18, 32)        0         
 g2D)                                                            
                                                                 
 dropout_9 (Dropout)         (None, 24, 18, 32)       

In [41]:
#테스트 데이터 정확도
# Evaluating the model on the training and testing set
score = model.evaluate(x_train, y_train, verbose=0)
print("Training Accuracy: ", score[1])

score = model.evaluate(x_test, y_test, verbose=0)
print("Testing Accuracy: ", score[1])

Training Accuracy:  1.0
Testing Accuracy:  1.0


In [43]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

wav, sr = librosa.load('/content/drive/MyDrive/CSE/newnewset/덕_123.wav')
mfcc = librosa.feature.mfcc(y=wav, sr=sr, n_mfcc=100)
pad_width = 75 - mfcc.shape[1]
mfcc = np.pad(mfcc, pad_width=((0,0), (0,pad_width)),mode='constant')
mfcc= np.expand_dims(mfcc, 0)

check = model.predict(mfcc)[0].tolist()
cc = check.index(max(check))
if cc==0: print('덕')
elif cc==1: print('덩')
elif cc==2: print('쿵')

#y_prob = model.predict(mfcc)
#predicted_vector = y_prob.argmax(axis=-1)
#predicted_class = le.inverse_transform(predicted_vector)
#print("The predicted class is:",predicted_class[0])

#[[덕],[덩],[쿵]]

1/1 [==============================] - 0s 294ms/step
덕


In [42]:
from keras.models import load_model

model.save('/content/drive/My Drive/CSE/1114_200.h5')